# Import Modules

In [1]:
# Import modules
import pandas as pd
import os

# Load Data

In [2]:
# Set up paths
RAW_PATH = "../data/raw/"
PROCESSED_PATH = "../data/processed/"

In [3]:
# Load and clean yield.csv
yield_df = pd.read_csv(os.path.join(RAW_PATH, "yield.csv"))[['Area', 'Item', 'Year', 'Value']]
yield_df['Value'] = pd.to_numeric(yield_df['Value'], errors='coerce')
yield_df = yield_df.dropna(subset=['Value']).rename(columns={'Value': 'Yield'})

# Load and clean area_harvested.csv
area_df = pd.read_csv(os.path.join(RAW_PATH, "area_harvested.csv"))[['Area', 'Item', 'Year', 'Value']]
area_df['Value'] = pd.to_numeric(area_df['Value'], errors='coerce')
area_df = area_df.dropna(subset=['Value']).rename(columns={'Value': 'Area_Harvested'})

# Load and clean production_quantity.csv
prod_df = pd.read_csv(os.path.join(RAW_PATH, "production_quantity.csv"))[['Area', 'Item', 'Year', 'Value']]
prod_df['Value'] = pd.to_numeric(prod_df['Value'], errors='coerce')
prod_df = prod_df.dropna(subset=['Value']).rename(columns={'Value': 'Production_Quantity'})

# Merge Data

In [4]:
# Harmonize country names
area_rename_map = {
    "Netherlands (Kingdom of the)": "Netherlands",
    "Republic of Moldova": "Moldova",
    "Slovakia": "Slovak Republic",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom"
}
for df in [yield_df, area_df, prod_df]:
    df['Area'] = df['Area'].replace(area_rename_map)

In [5]:
# Merge the three main datasets
merged_df = pd.merge(yield_df, area_df, on=['Area', 'Item', 'Year'], how='inner')
merged_df = pd.merge(merged_df, prod_df, on=['Area', 'Item', 'Year'], how='inner')

In [6]:
# Drop obsolete countries
drop_countries = [
    "Belgium-Luxembourg", "Czechoslovakia", "Serbia and Montenegro", "USSR", "Yugoslav SFR"
]
merged_df = merged_df[~merged_df['Area'].isin(drop_countries)]

In [7]:
# Remove year 2023 (not present in all datasets)
merged_df = merged_df[merged_df['Year'] <= 2022]

In [8]:
# Load and process features.csv
features_df = pd.read_csv(os.path.join(RAW_PATH, "features.csv"))
features_long = features_df.melt(
    id_vars=["REF_AREA_LABEL", "INDICATOR", "INDICATOR_LABEL"],
    var_name="Year",
    value_name="Value"
)
features_long = features_long.rename(columns={"REF_AREA_LABEL": "Area"})
features_long['Year'] = pd.to_numeric(features_long['Year'], errors='coerce')
features_long = features_long.dropna(subset=['Year', 'Value'])
features_long['Year'] = features_long['Year'].astype(int)

In [9]:
# Pivot features
features_pivot = features_long.pivot_table(
    index=["Area", "Year"],
    columns="INDICATOR",
    values="Value",
    aggfunc="first"
).reset_index()

In [10]:
# Merge with features
final_df = pd.merge(merged_df, features_pivot, on=["Area", "Year"], how="left")

# Save Merged Dataset

In [11]:
# Uppercase headers and save
final_df.columns = [col.upper() for col in final_df.columns]
os.makedirs(PROCESSED_PATH, exist_ok=True)

In [12]:
# Save to CSV
final_df.to_csv(os.path.join(PROCESSED_PATH, "merged_dataset.csv"), index=False)

In [13]:
# Save to Excel
final_df.to_excel(os.path.join(PROCESSED_PATH, "merged_dataset.xlsx"), index=False)

In [14]:
# Preview
print("Final merged dataset preview:")
display(final_df.head())

Final merged dataset preview:


,AREA,ITEM,YEAR,YIELD,AREA_HARVESTED,PRODUCTION_QUANTITY,WB_CCKP_CDD,WB_CCKP_CDD65,WB_CCKP_CSDI,WB_CCKP_CWD,...,WB_CCKP_TASMIN,WB_CCKP_TNN,WB_CCKP_TR,WB_CCKP_TR23,WB_CCKP_TR26,WB_CCKP_TR29,WB_CCKP_TR32,WB_CCKP_TX84RR,WB_CCKP_TXX,WB_CCKP_WSDI
0,Albania,Barley,1990,1000.0,5000.0,5000.0,27.05,548.49,5.6,14.53,...,12.5,-8.07,66.02,25.22,2.65,0.02,0.0,0.05,24.78,21.31
1,Albania,Barley,1991,923.1,3600.0,3323.0,23.78,492.26,33.37,9.44,...,11.21,-13.93,60.51,21.68,2.48,0.0,0.0,0.03,24.22,0.38
2,Albania,Barley,1992,1380.2,4035.0,5569.0,29.91,579.35,7.62,10.14,...,11.83,-11.56,66.79,27.46,5.56,0.11,0.0,0.08,25.25,0.0
3,Albania,Barley,1993,1332.4,3117.0,4153.0,24.83,629.48,7.81,9.02,...,11.99,-11.03,73.15,30.26,6.19,0.26,0.0,0.1,26.22,6.07
4,Albania,Barley,1994,2352.3,3826.0,9000.0,24.24,810.85,0.0,11.43,...,13.06,-6.09,87.27,42.15,11.38,0.88,0.08,0.15,27.95,9.24
